In [3]:
import os
import pandas as pd
import torch

from fairseq_signals.utils.store import MemmapReader

/home/aa2650/playground/ECG-FM/virtualenv/lib/python3.9/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [1]:
root = '/home/aa2650/datasets/code_15/subset'
experiment_root = '/home/aa2650/datasets/code_15/experiments/subset'
fairseq_signals_root = '/home/aa2650/playground/fairseq-signals'
fairseq_signals_root = fairseq_signals_root.rstrip('/')
fairseq_signals_root

data_split = "80-10-10"

In [19]:
each_experiment_path = os.path.join(experiment_root, data_split)
os.makedirs(each_experiment_path, exist_ok=True)
print(each_experiment_path)

/home/aa2650/datasets/code_15/experiments/subset/80-10-10


The segmented split must be saved with absolute file paths, so we will update the current relative file paths accordingly.

In [6]:
# The bottom part only needs to be run once
segmented_path = f'/home/aa2650/datasets/code_15/subset/segmented_split.csv'
segmented_split = pd.read_csv(segmented_path,
    index_col='idx',
)
# segmented_split['path'] = ('/home/aa2650/datasets/code_15/segmented/') + segmented_split['path']
# segmented_split.to_csv(os.path.join('/home/aa2650/datasets/code_15/segmented_split.csv'))

Run the follow commands togenerate the `test.tsv` file used for inference.

In [ ]:
model_path = f"/home/aa2650/playground/ECG-FM/experiments/subset/80-10-10/checkpoint21.pt"

assert os.path.isfile(f"{model_path}")


In [75]:
# Copy

# inference_cmd = f"""fairseq-hydra-inference \\
#     task.data="/home/aa2650/datasets/code_15/manifests/" \\
#     common_eval.path="/home/aa2650/playground/ECG-FM/experiments/0.01training/checkpoint40.pt" \\
#     common_eval.results_path="{each_experiment_path}" \\
#     model.num_labels=26 \\
#     dataset.valid_subset="test" \\
#     dataset.batch_size=10 \\
#     dataset.num_workers=3 \\
#     dataset.disable_validation=false \\
#     distributed_training.distributed_world_size=1 \\
#     distributed_training.find_unused_parameters=True \\
#     --config-dir "/home/aa2650/playground/ECG-FM/ckpts/" \\
#     --config-name physionet_finetuned
# """

# os.system(inference_cmd)

In [76]:
inference_cmd = f"""fairseq-hydra-inference \\
    task.data="/home/aa2650/datasets/code_15/subset/manifests" \\
    common_eval.path="{model_path}" \\
    common_eval.results_path="{each_experiment_path}" \\
    model.num_labels=8 \\
    dataset.valid_subset="test" \\
    dataset.batch_size=10 \\
    dataset.num_workers=3 \\
    dataset.disable_validation=false \\
    distributed_training.distributed_world_size=1 \\
    distributed_training.find_unused_parameters=True \\
    --config-dir "/home/aa2650/playground/ECG-FM/ckpts/" \\
    --config-name physionet_finetuned
"""

os.system(inference_cmd)

/home/aa2650/playground/ECG-FM/virtualenv/lib/python3.9/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


[2025-03-29 16:47:07,514][fairseq_cli.inference][INFO] - loading model from /home/aa2650/playground/ECG-FM/experiments/subset/80-10-10/checkpoint17.pt
[2025-03-29 16:47:09,862][fairseq_signals.utils.checkpoint_utils][INFO] - Loaded a checkpoint in 2.35s
[2025-03-29 16:47:09,864][fairseq_cli.inference][INFO] - num. shared model params: 90,377,862 (num. trained: 90,377,862)
[2025-03-29 16:47:09,865][fairseq_cli.inference][INFO] - num. expert model params: 0 (num. trained: 0)
[2025-03-29 16:47:10,267][fairseq_cli.inference][INFO] - {'_name': None,
 'checkpoint': {'_name': None, 'save_dir': '/home/aa2650/playground/ECG-FM/experiments/subset/80-10-10', 'restore_file': 'checkpoint_last.pt', 'finetune_from_model': None, 'reset_dataloader': False, 'reset_lr_scheduler': False, 'reset_meters': False, 'reset_optimizer': False, 'optimizer_overrides': '{}', 'save_interval': 1, 'save_interval_updates': 0, 'keep_interval_updates': -1, 'keep_interval_updates_pattern': -1, 'keep_last_epochs': 0, 'keep_

0

In [77]:
assert os.path.isfile(f"{each_experiment_path}/outputs_test.npy")
assert os.path.isfile(f"{each_experiment_path}/outputs_test_header.pkl")

## 4. Interpret results

The logits are ordered same as the samples in the manifest and labels in the label definition.

### Get predictions on PhysioNet 2021 labels

In [78]:
# os.path.join('/home/aa2650/playground/ECG-FM/data/code_15/labels/label_def.csv'),

code15_label_def = pd.read_csv(
    os.path.join('/home/aa2650/datasets/code_15/subset/label_def.csv'),
     index_col='name',
)
code15_label_names = code15_label_def.index
code15_label_def

,pos_count_all,pos_percent_all
name,,
RBBB,9672,0.224518
LBBB,6026,0.139883
SB,5605,0.130110
ST,7584,0.176049
AF,7033,0.163258
normal_ecg,9500,0.220525


In [79]:
# Load the array of computed logits
logits = MemmapReader.from_header(f"{each_experiment_path}/outputs_test.npy")[:]
logits.shape

(5873, 6)

In [80]:
# Construct predictions from logits
pred = pd.DataFrame(
    torch.sigmoid(torch.tensor(logits)).numpy(),
    columns=code15_label_names,
)

# Join in sample information
pred = segmented_split.reset_index().join(pred, how='left').set_index('idx')
pred

,save_file,split,path,sample_size,RBBB,LBBB,SB,ST,AF,normal_ecg
idx,,,,,,,,,,
35604,code_15_1000182.mat,train,/home/aa2650/datasets/code_15/subset/segmented...,2500,1.116773e-03,9.986595e-01,0.000251,0.000033,0.001609,9.793499e-08
40621,code_15_1000220.mat,train,/home/aa2650/datasets/code_15/subset/segmented...,2500,5.291291e-04,9.983504e-01,0.000304,0.000034,0.002091,6.110803e-08
6332,code_15_1000229.mat,train,/home/aa2650/datasets/code_15/subset/segmented...,2500,3.523358e-04,9.999589e-01,0.000193,0.000015,0.000311,1.011731e-08
6332,code_15_1000229.mat,train,/home/aa2650/datasets/code_15/subset/segmented...,2500,9.558569e-07,2.592195e-08,0.000055,0.999961,0.000308,1.449786e-08
2496,code_15_1000253.mat,train,/home/aa2650/datasets/code_15/subset/segmented...,2500,8.441068e-05,8.327872e-05,0.000431,0.000008,0.000183,9.989073e-01
...,...,...,...,...,...,...,...,...,...,...
22837,code_15_999826.mat,train,/home/aa2650/datasets/code_15/subset/segmented...,2500,NaN,NaN,NaN,NaN,NaN,NaN
5263,code_15_999827.mat,train,/home/aa2650/datasets/code_15/subset/segmented...,2500,NaN,NaN,NaN,NaN,NaN,NaN
5263,code_15_999827.mat,train,/home/aa2650/datasets/code_15/subset/segmented...,2500,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
# Perform a (crude) thresholding of 0.5 for all labels
pred_thresh = pred.copy()
pred_thresh[code15_label_names] = pred_thresh[code15_label_names] > 0.5

# Construct a readable column of predicted labels for each sample
pred_thresh['labels'] = pred_thresh[code15_label_names].apply(
    lambda row: ', '.join(row.index[row]),
    axis=1,
)
pred_thresh['labels']

idx
35604          LBBB
40621          LBBB
6332           LBBB
6332             ST
2496     normal_ecg
            ...    
22837              
5263               
5263               
15475              
15475              
Name: labels, Length: 58984, dtype: object

### Map predictions to CODE-15 labels

In [82]:
code_15_label_def = pd.read_csv("/home/aa2650/playground/ECG-FM/data/code_15/labels/label_def.csv",
     index_col='name',
)
code_15_label_names = code_15_label_def.index
code_15_label_def

,pos_count_all,pos_percent_all
name,,
is_male,138528,0.402691
1dAVb,5699,0.016567
RBBB,9652,0.028058
LBBB,6011,0.017474
SB,5588,0.016244
ST,7571,0.022008
AF,7008,0.020372
normal_ecg,134497,0.390973


In [83]:
label_mapping = {
    'RBBB': 'RBBB',
    'LBBB': 'LBBB',
    'SB': 'SB',
    'ST': 'ST',
    'AF': 'AF',
    'normal_ecg': 'normal_ecg'
}

code15_label_def['name_mapped'] = code15_label_def.index.map(label_mapping)
code15_label_def

,pos_count_all,pos_percent_all,name_mapped
name,,,
RBBB,9672,0.224518,RBBB
LBBB,6026,0.139883,LBBB
SB,5605,0.130110,SB
ST,7584,0.176049,ST
AF,7033,0.163258,AF
normal_ecg,9500,0.220525,normal_ecg


In [84]:
pred_mapped = pred.copy()
pred_mapped.drop(set(code15_label_names) - set(label_mapping.keys()), axis=1, inplace=True)
pred_mapped.rename(label_mapping, axis=1, inplace=True)
pred_mapped

,save_file,split,path,sample_size,RBBB,LBBB,SB,ST,AF,normal_ecg
idx,,,,,,,,,,
35604,code_15_1000182.mat,train,/home/aa2650/datasets/code_15/subset/segmented...,2500,1.116773e-03,9.986595e-01,0.000251,0.000033,0.001609,9.793499e-08
40621,code_15_1000220.mat,train,/home/aa2650/datasets/code_15/subset/segmented...,2500,5.291291e-04,9.983504e-01,0.000304,0.000034,0.002091,6.110803e-08
6332,code_15_1000229.mat,train,/home/aa2650/datasets/code_15/subset/segmented...,2500,3.523358e-04,9.999589e-01,0.000193,0.000015,0.000311,1.011731e-08
6332,code_15_1000229.mat,train,/home/aa2650/datasets/code_15/subset/segmented...,2500,9.558569e-07,2.592195e-08,0.000055,0.999961,0.000308,1.449786e-08
2496,code_15_1000253.mat,train,/home/aa2650/datasets/code_15/subset/segmented...,2500,8.441068e-05,8.327872e-05,0.000431,0.000008,0.000183,9.989073e-01
...,...,...,...,...,...,...,...,...,...,...
22837,code_15_999826.mat,train,/home/aa2650/datasets/code_15/subset/segmented...,2500,NaN,NaN,NaN,NaN,NaN,NaN
5263,code_15_999827.mat,train,/home/aa2650/datasets/code_15/subset/segmented...,2500,NaN,NaN,NaN,NaN,NaN,NaN
5263,code_15_999827.mat,train,/home/aa2650/datasets/code_15/subset/segmented...,2500,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
pred_thresh_mapped = pred_thresh.copy()
pred_thresh_mapped.drop(set(code15_label_names) - set(label_mapping.keys()), axis=1, inplace=True)
pred_thresh_mapped.rename(label_mapping, axis=1, inplace=True)
pred_thresh_mapped['predicted'] = pred_thresh_mapped[label_mapping.values()].apply(
    lambda row: ', '.join(row.index[row]),
    axis=1,
)
pred_thresh_mapped

,save_file,split,path,sample_size,RBBB,LBBB,SB,ST,AF,normal_ecg,labels,predicted
idx,,,,,,,,,,,,
35604,code_15_1000182.mat,train,/home/aa2650/datasets/code_15/subset/segmented...,2500,False,True,False,False,False,False,LBBB,LBBB
40621,code_15_1000220.mat,train,/home/aa2650/datasets/code_15/subset/segmented...,2500,False,True,False,False,False,False,LBBB,LBBB
6332,code_15_1000229.mat,train,/home/aa2650/datasets/code_15/subset/segmented...,2500,False,True,False,False,False,False,LBBB,LBBB
6332,code_15_1000229.mat,train,/home/aa2650/datasets/code_15/subset/segmented...,2500,False,False,False,True,False,False,ST,ST
2496,code_15_1000253.mat,train,/home/aa2650/datasets/code_15/subset/segmented...,2500,False,False,False,False,False,True,normal_ecg,normal_ecg
...,...,...,...,...,...,...,...,...,...,...,...,...
22837,code_15_999826.mat,train,/home/aa2650/datasets/code_15/subset/segmented...,2500,False,False,False,False,False,False,,
5263,code_15_999827.mat,train,/home/aa2650/datasets/code_15/subset/segmented...,2500,False,False,False,False,False,False,,
5263,code_15_999827.mat,train,/home/aa2650/datasets/code_15/subset/segmented...,2500,False,False,False,False,False,False,,


### Compare predicted CODE-15 to actual

In [86]:
true_labels = pd.read_csv(os.path.join('/home/aa2650/datasets/code_15/subset/labels.csv'), index_col='idx')
true_labels['actual'] = true_labels[label_mapping.values()].apply(
    lambda row: ', '.join(row.index[row]),
    axis=1,
)
true_labels

,RBBB,LBBB,SB,ST,AF,normal_ecg,actual
idx,,,,,,,
0,False,False,False,False,True,False,AF
1,False,False,False,False,True,False,AF
2,False,False,True,False,False,False,SB
3,True,False,False,False,False,False,RBBB
4,False,False,False,False,True,False,AF
...,...,...,...,...,...,...,...
43074,False,False,False,False,False,True,normal_ecg
43075,False,False,False,False,False,True,normal_ecg
43076,False,False,False,False,False,True,normal_ecg


In [87]:
# Visualize predicted and actual labels side-by-side
pred_thresh_mapped[['predicted']].join(true_labels[['actual']], how='left')

# Calculate accuracy
# Compare predicted and actual labels
comparison = pred_thresh_mapped[['predicted']].join(true_labels[['actual']], how='left')

# Calculate overall accuracy (exact match)
accuracy = (comparison['predicted'] == comparison['actual']).mean()
print(f"Overall accuracy: {accuracy:.2%}")

Overall accuracy: 1.61%


In [88]:
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MultiLabelBinarizer

y_true_str = comparison['actual']
y_pred_str = comparison['predicted']

y_true_list = [labels.split(", ") for labels in y_true_str]
y_pred_list = [labels.split(", ") for labels in y_pred_str]

mlb = MultiLabelBinarizer()
y_true_bin = mlb.fit_transform(y_true_list)
y_pred_bin = mlb.transform(y_pred_list)

class_names = mlb.classes_

results = []

for i, cls_name in enumerate(class_names):
    y_true_col = y_true_bin[:, i]
    y_pred_col = y_pred_bin[:, i]
    
    tn, fp, fn, tp = confusion_matrix(y_true_col, y_pred_col).ravel()
    total = tp + tn + fp + fn
    prevalence = (tp + fn) / total
    sensitivity = tp / (tp + fn)
    precision = tp / (tp + fp)
    f1 = 2 * (precision * sensitivity) / (precision + sensitivity)
    specificity = tn / (tn + fp)
    npv = tn / (tn + fn)
    accuracy = (tp + tn) / total

    results.append({
        'class': cls_name,
        'prevalence': round(prevalence, 3),
        'precision': round(precision, 3),
        'f1': round(f1, 3),
        'sensitivity': round(sensitivity, 3),
        'specificity': round(specificity, 3),
        'accuracy': round(accuracy, 3),
    })

metrics_df = pd.DataFrame(results)
print(metrics_df)

        class  prevalence  precision     f1  sensitivity  specificity  \
0          AF       0.164      0.137  0.023        0.013        0.984   
1        LBBB       0.143      0.138  0.025        0.014        0.986   
2        RBBB       0.216      0.221  0.039        0.021        0.979   
3          SB       0.130      0.131  0.026        0.015        0.986   
4          ST       0.173      0.173  0.031        0.017        0.983   
5  normal_ecg       0.228      0.277  0.048        0.026        0.980   

   accuracy  
0     0.825  
1     0.847  
2     0.772  
3     0.859  
4     0.815  
5     0.763  


/home/aa2650/playground/ECG-FM/virtualenv/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:909: UserWarning: unknown class(es) [''] will be ignored
  warnings.warn(
